In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, confusion_matrix, roc_curve, auc)
import shap
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# 1. VERİ SETİNİN YÜKLENMESI
# ============================================================================
print("="*80)
print("1. VERİ SETİNİN YÜKLENMESI")
print("="*80)

wine = load_wine()
X = wine.data
y = wine.target

df = pd.DataFrame(X, columns=wine.feature_names)
df['target'] = y

print("\nVeri seti şekli:", df.shape)
print("\nİlk 5 satır:")
print(df.head())

print("\nHedref değişken dağılımı:")
print(df['target'].value_counts().sort_index())

# ============================================================================
# 2. VERİ SETİ KALİTE KONTROLLERI
# ============================================================================
print("\n" + "="*80)
print("2. VERİ SETİ KALİTE KONTROLLERI")
print("="*80)

print("\n2.1 Eksik Değer Analizi:")
missing = df.isnull().sum()
print(missing)
print(f"Toplam eksik değer: {missing.sum()}")

print("\n2.2 Aykırı Değer Analizi (IQR Yöntemi):")
def detect_outliers_iqr(df_temp):
    outlier_indices = []
    numeric_cols = df_temp.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        Q1 = df_temp[col].quantile(0.25)
        Q3 = df_temp[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outlier_idx = df_temp[(df_temp[col] < lower) | (df_temp[col] > upper)].index
        outlier_indices.extend(outlier_idx)
    return list(set(outlier_indices))

outlier_indices = detect_outliers_iqr(df.drop('target', axis=1))
print(f"Aykırı değer sayısı: {len(outlier_indices)} ({len(outlier_indices)/len(df)*100:.2f}%)")
print("Aykırı değerlerin modele etkisi: Sınıflandırma performansını düşürebilir, ancak Wine veri seti ")
print("doğal olarak dengeli olduğundan etkisi minimal olacaktır.")

print("\n2.3 Veri Tipi ve Dağılım İncelemesi:")
print(f"Sayısal değişken sayısı: {len(df.select_dtypes(include=[np.number]).columns)}")
print(f"Kategorik değişken sayısı: {len(df.select_dtypes(include=['object']).columns)}")
print("\nVeri tipleri:")
print(df.dtypes)

# ============================================================================
# 3. KEŞİFSEL VERİ ANALİZİ (EDA)
# ============================================================================
print("\n" + "="*80)
print("3. KEŞİFSEL VERİ ANALİZİ (EDA)")
print("="*80)

print("\n3.1 İstatistiksel Özellikler:")
stats_df = df.drop('target', axis=1).describe().T
stats_df['Q1'] = df.drop('target', axis=1).quantile(0.25)
stats_df['Q3'] = df.drop('target', axis=1).quantile(0.75)
print(stats_df[['mean', '50%', 'min', 'max', 'std', 'Q1', 'Q3']])

print("\n3.2 Korelasyon Matrisi:")
corr_matrix = df.drop('target', axis=1).corr()
print("En yüksek korelasyonlu 3 çift özellik:")
corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], 
                          abs(corr_matrix.iloc[i, j])))
top_corr = sorted(corr_pairs, key=lambda x: x[2], reverse=True)[:3]
for feat1, feat2, corr in top_corr:
    print(f"{feat1} <-> {feat2}: {corr:.4f}")

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0, ax=ax)
plt.title('Korelasyon Matrisi')
plt.tight_layout()
plt.savefig('correlation_matrix.png', dpi=100)
plt.close()

print("\n3.3 Boxplot Analizi:")
fig, axes = plt.subplots(4, 4, figsize=(16, 12))
for idx, col in enumerate(df.drop('target', axis=1).columns):
    ax = axes[idx // 4, idx % 4]
    ax.boxplot(df[col])
    ax.set_title(col, fontsize=8)
plt.tight_layout()
plt.savefig('boxplots.png', dpi=100)
plt.close()
print("Boxplot görselleri kaydedildi.")

# ============================================================================
# 4. VERİ ÖLÇEKLENDIRME
# ============================================================================
print("\n" + "="*80)
print("4. VERİ ÖLÇEKLENDIRME (StandardScaler)")
print("="*80)

X_scaled = StandardScaler().fit_transform(X)
print(f"Ölçeklendirme tamamlandı. Şekil: {X_scaled.shape}")

# ============================================================================
# 5. VERİ SETİNİN BÖLÜNMESI
# ============================================================================
print("\n" + "="*80)
print("5. VERİ SETİNİN BÖLÜNMESI")
print("="*80)

X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp)

print(f"Training seti: {len(X_train)} (%{len(X_train)/len(X_scaled)*100:.1f})")
print(f"Validation seti: {len(X_val)} (%{len(X_val)/len(X_scaled)*100:.1f})")
print(f"Test seti: {len(X_test)} (%{len(X_test)/len(X_scaled)*100:.1f})")

# ============================================================================
# 6. BOYUT İNDİRGEME
# ============================================================================
print("\n" + "="*80)
print("6. BOYUT İNDİRGEME")
print("="*80)

print("\n6.1 PCA (Principal Component Analysis):")
pca = PCA()
pca.fit(X_train)
exp_var = pca.explained_variance_ratio_
cum_var = np.cumsum(exp_var)
mean_var = exp_var.mean()
n_components_pca = np.argmax(cum_var >= cum_var[-1] * 0.95) + 1

print(f"PCA bileşen sayısı (95% varyans): {n_components_pca}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
ax1.plot(range(1, len(exp_var)+1), exp_var, 'bo-')
ax1.set_xlabel('PCA Bileşen Sayısı')
ax1.set_ylabel('Explained Variance Ratio')
ax1.set_title('PCA Explained Variance')
ax1.grid()

ax2.plot(range(1, len(cum_var)+1), cum_var, 'ro-')
ax2.axhline(y=0.95, color='g', linestyle='--', label='95% Threshold')
ax2.set_xlabel('PCA Bileşen Sayısı')
ax2.set_ylabel('Cumulative Explained Variance')
ax2.set_title('PCA Cumulative Explained Variance')
ax2.legend()
ax2.grid()
plt.tight_layout()
plt.savefig('pca_variance.png', dpi=100)
plt.close()

pca = PCA(n_components=n_components_pca)
X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

# 2D visualization
pca_2d = PCA(n_components=2)
X_train_pca_2d = pca_2d.fit_transform(X_train)

fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(X_train_pca_2d[:, 0], X_train_pca_2d[:, 1], c=y_train, 
                     cmap='viridis', alpha=0.6)
ax.set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]:.2%})')
ax.set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]:.2%})')
ax.set_title('PCA - Sınıfların 2D Ayrışması')
plt.colorbar(scatter, ax=ax, label='Sınıf')
plt.tight_layout()
plt.savefig('pca_2d.png', dpi=100)
plt.close()

print("\n6.2 LDA (Linear Discriminant Analysis):")
lda = LinearDiscriminantAnalysis(n_components=2)
X_train_lda = lda.fit_transform(X_train, y_train)
X_val_lda = lda.transform(X_val)
X_test_lda = lda.transform(X_test)

fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(X_train_lda[:, 0], X_train_lda[:, 1], c=y_train, 
                     cmap='viridis', alpha=0.6)
ax.set_xlabel('LD1')
ax.set_ylabel('LD2')
ax.set_title('LDA - Sınıfların 2D Ayrışması')
plt.colorbar(scatter, ax=ax, label='Sınıf')
plt.tight_layout()
plt.savefig('lda_2d.png', dpi=100)
plt.close()

# ============================================================================
# 7 & 8. MODELLERİN KURULMASI VE VALİDASYON
# ============================================================================
print("\n" + "="*80)
print("7 & 8. MAKİNE ÖĞRENMESI MODELLERİ VE VALİDASYON")
print("="*80)

def get_algorithm_instance(algo_name):
    """Her veri versiyonu için yeni model instance'ı oluştur"""
    if algo_name == 'Logistic Regression':
        return LogisticRegression(max_iter=1000, random_state=42)
    elif algo_name == 'Decision Tree':
        return DecisionTreeClassifier(random_state=42)
    elif algo_name == 'Random Forest':
        return RandomForestClassifier(n_estimators=100, random_state=42)
    elif algo_name == 'XGBoost':
        return XGBClassifier(random_state=42, verbose=0)
    elif algo_name == 'Naive Bayes':
        return GaussianNB()

algorithm_names = ['Logistic Regression', 'Decision Tree', 'Random Forest', 'XGBoost', 'Naive Bayes']

data_versions = {
    'Ham Veri': (X_train, X_val, X_test),
    'PCA': (X_train_pca, X_val_pca, X_test_pca),
    'LDA': (X_train_lda, X_val_lda, X_test_lda)
}

results = []
models_dict = {}

for data_name, (X_tr, X_v, X_te) in data_versions.items():
    for algo_name in algorithm_names:
        model = get_algorithm_instance(algo_name)
        model.fit(X_tr, y_train)
        y_pred = model.predict(X_v)
        y_pred_proba = model.predict_proba(X_v)[:, 1] if len(np.unique(y)) == 2 else model.predict_proba(X_v).max(axis=1)
        
        try:
            roc_auc = roc_auc_score(y_val, y_pred_proba, multi_class='ovr')
        except:
            roc_auc = 0.0
        
        results.append({
            'Veri': data_name,
            'Algoritma': algo_name,
            'Accuracy': accuracy_score(y_val, y_pred),
            'Precision': precision_score(y_val, y_pred, average='weighted', zero_division=0),
            'Recall': recall_score(y_val, y_pred, average='weighted', zero_division=0),
            'F1': f1_score(y_val, y_pred, average='weighted', zero_division=0),
            'ROC-AUC': roc_auc
        })
        
        models_dict[f"{data_name}_{algo_name}"] = model

results_df = pd.DataFrame(results)
print("\nTüm Modellerin Validation Performansı:")
print(results_df.to_string(index=False))

best_idx = results_df['Accuracy'].idxmax()
best_result = results_df.loc[best_idx]
print(f"\nEn İyi Model: {best_result['Veri']} - {best_result['Algoritma']}")
print(f"Accuracy: {best_result['Accuracy']:.4f}")

# ============================================================================
# 9. EN İYİ MODELİN TEST ÜZERİNDE DEĞERLENDİRİLMESİ
# ============================================================================
print("\n" + "="*80)
print("9. EN İYİ MODELİN TEST ÜZERİNDE DEĞERLENDİRİLMESİ")
print("="*80)

best_model_name = f"{best_result['Veri']}_{best_result['Algoritma']}"
best_model = models_dict[best_model_name]

if best_result['Veri'] == 'Ham Veri':
    X_test_best = X_test
elif best_result['Veri'] == 'PCA':
    X_test_best = X_test_pca
else:
    X_test_best = X_test_lda

y_test_pred = best_model.predict(X_test_best)
y_test_pred_proba = best_model.predict_proba(X_test_best)

print(f"\n9.1 Performans Metrikleri:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_test_pred, average='weighted', zero_division=0):.4f}")
print(f"Recall: {recall_score(y_test, y_test_pred, average='weighted', zero_division=0):.4f}")
print(f"F1-Score: {f1_score(y_test, y_test_pred, average='weighted', zero_division=0):.4f}")

try:
    roc_auc_test = roc_auc_score(y_test, y_test_pred_proba, multi_class='ovr')
    print(f"ROC-AUC: {roc_auc_test:.4f}")
except:
    print("ROC-AUC: Hesaplanmadı")

print(f"\n9.2 Confusion Matrix:")
cm = confusion_matrix(y_test, y_test_pred)
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_xlabel('Tahmin')
ax.set_ylabel('Gerçek')
ax.set_title('Confusion Matrix')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=100)
plt.close()

print(f"\n9.3 ROC Eğrisi:")
fig, ax = plt.subplots(figsize=(10, 8))
for class_idx in range(len(np.unique(y))):
    fpr, tpr, _ = roc_curve((y_test == class_idx).astype(int), 
                            y_test_pred_proba[:, class_idx])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, label=f'Sınıf {class_idx} (AUC={roc_auc:.3f})')
ax.plot([0, 1], [0, 1], 'k--', label='Random')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Eğrisi')
ax.legend()
ax.grid()
plt.tight_layout()
plt.savefig('roc_curve.png', dpi=100)
plt.close()

# ============================================================================
# 10. SHAP AÇIKLANABILIRLK ANALİZİ
# ============================================================================
print("\n" + "="*80)
print("10. SHAP AÇIKLANABILIRLK ANALİZİ")
print("="*80)

print("\n10.1 En İyi Model için SHAP Analizi:")

if best_result['Veri'] == 'Ham Veri':
    X_val_best = X_val
    feature_names = wine.feature_names
elif best_result['Veri'] == 'PCA':
    X_val_best = X_val_pca
    feature_names = [f'PC_{i+1}' for i in range(X_val_pca.shape[1])]
else:
    X_val_best = X_val_lda
    feature_names = [f'LD_{i+1}' for i in range(X_val_lda.shape[1])]

if isinstance(best_model, (DecisionTreeClassifier, RandomForestClassifier, XGBClassifier)):
    explainer = shap.TreeExplainer(best_model)
else:
    explainer = shap.KernelExplainer(best_model.predict, X_val_best[:100])

shap_values = explainer.shap_values(X_val_best)

fig = plt.figure(figsize=(12, 8))
if isinstance(shap_values, list):
    shap.summary_plot(shap_values[1], X_val_best, feature_names=feature_names, show=False)
else:
    shap.summary_plot(shap_values, X_val_best, feature_names=feature_names, show=False)
plt.tight_layout()
plt.savefig('shap_summary.png', dpi=100, bbox_inches='tight')
plt.close()

fig = plt.figure(figsize=(10, 6))
if isinstance(shap_values, list):
    shap.summary_plot(shap_values[1], X_val_best, feature_names=feature_names, plot_type="bar", show=False)
else:
    shap.summary_plot(shap_values, X_val_best, feature_names=feature_names, plot_type="bar", show=False)
plt.tight_layout()
plt.savefig('shap_bar.png', dpi=100, bbox_inches='tight')
plt.close()

print("SHAP görselleri kaydedildi.")
print("En önemli özellikler model tarafından belirlenen sıralamadır.")

print("\n" + "="*80)
print("PROJE TAMAMLANDI")
print("="*80)

1. VERİ SETİNİN YÜKLENMESI

Veri seti şekli: (178, 14)

İlk 5 satır:
   alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0    14.23        1.71  2.43               15.6      127.0           2.80   
1    13.20        1.78  2.14               11.2      100.0           2.65   
2    13.16        2.36  2.67               18.6      101.0           2.80   
3    14.37        1.95  2.50               16.8      113.0           3.85   
4    13.24        2.59  2.87               21.0      118.0           2.80   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.

  0%|          | 0/18 [00:00<?, ?it/s]

SHAP görselleri kaydedildi.
En önemli özellikler model tarafından belirlenen sıralamadır.

PROJE TAMAMLANDI
